# Data Processing

## Imports

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

## Data Exploration

In [ ]:
df = pd.read_csv('../data/sp500_dataset.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Adj Close
0,2020-08-10 00:00:00-04:00,109.652317,110.796560,107.120382,109.776482,212403600.0,0.0,0.0,AAPL,NaN
1,2020-08-11 00:00:00-04:00,109.038826,109.537906,106.251258,106.511757,187902400.0,0.0,0.0,AAPL,NaN
2,2020-08-12 00:00:00-04:00,107.604859,110.309652,107.410097,110.051590,165598000.0,0.0,0.0,AAPL,NaN
3,2020-08-13 00:00:00-04:00,111.434419,113.004709,110.945071,111.999237,210082000.0,0.0,0.0,AAPL,NaN
4,2020-08-14 00:00:00-04:00,111.823959,111.989507,110.085683,111.899429,165565200.0,0.0,0.0,AAPL,NaN


Lets check how much data each company has.

In [27]:
sub_df = df.groupby('Ticker').size().sort_values(ascending=False)
print(sub_df.unique())
sub_df[-12:]

[1256 1241 1240 1230 1220 1170  892  663  568  463  344  343  219]


Ticker
HES     1241
ANSS    1240
JNPR    1230
PLTR    1220
ABNB    1170
CEG      892
GEHC     663
KVUE     568
VLTO     463
SOLV     344
GEV      343
AMTM     219
dtype: int64